In [ ]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 8.0 MB/s eta 0:00:00


In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 65.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 98.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.6 MB/s eta 0:00:00


In [ ]:
!pip install farasapy==0.0.14
!pip install pyarabic==0.6.14
!git clone https://github.com/aub-mind/arabert
!pip install emoji==1.6.1
!pip install sentencepiece==0.1.96

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 2.6 MB/s eta 0:00:00
Cloning into 'arabert'...
remote: Enumerating objects: 600, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 600 (delta 38), reused 45 (delta 30), pack-reused 535
Receiving objects: 100% (600/600), 9.14 MiB | 13.06 MiB/s, done.
Resolving deltas: 100% (339/339), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.0/170.0 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-1.6.1-py3-none-any.whl size=169295 sha256=1d309975068e493f5b6ad117ce5805eb4969c23ceb1efcec144dd0dd57a1aa6e
  Stored in directory: /root/.cache/pip/wheels/3d/c9/af/02caa5725634f27f4e2e43852f67fc9069d014038b236a827e
Successfully built emoji
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 6.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from typing import List
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split

In [ ]:
class CustomDataset:
    def __init__(
        self,
        name: str,
        train: List[pd.DataFrame],
        test: List[pd.DataFrame],
        label_list: List[str],
    ):
        """Class to hold and structure datasets.

        Args:

        name (str): holds the name of the dataset so we can select it later
        train (List[pd.DataFrame]): holds training pandas dataframe with 2 columns ["text","label"]
        test (List[pd.DataFrame]): holds testing pandas dataframe with 2 columns ["text","label"]
        label_list (List[str]): holds the list  of labels
        """
        self.name = name
        self.train = train
        self.test = test
        self.label_list = label_list

In [ ]:
data = pd.read_excel("1not0.xlsx")
data.head

<bound method NDFrame.head of                                                    text  Label
0     وين كان عقلي يوم سجلت رقمي ليدر بكل المواد؟ \n...      0
1     @alialkhalede مرات أگول خلي انتحر وفتك بس أفكر...      0
2     افكر انتحر ببث مباشر لكن محتار \nبث مباشر فيس ...      1
3     لم اجي اشتكيلك من سلبياتي عشان تعباني .. يا ته...      1
4     @Tulip_rose2 @Be_Cool68 🤔 يعني .. جني تسرعت \n...      0
...                                                 ...    ...
5714  @Dr_Alchemist1 الفتره دى تحديدا احساسى بالغربه...      1
5715    @k_asiri4 🤣🤣🤣🤣 وإذا كان كل يوم وجها لوجة  انتحر      0
5716   @alshmaly999 مااردك انت انتحر الاول وانا وراك🤣🤣🤣      1
5717    @22334455zzz شغلة الدكتور الى قتلوه وصاحو انتحر      0
5718  @Amarin123456 لكن الملحدون طوابير على مشانق ال...      0

[5719 rows x 2 columns]>

In [ ]:
DATA_COLUMN = "text"
LABEL_COLUMN = "label"
df_ASTD_UN =data

df_ASTD_UN.columns = [DATA_COLUMN, LABEL_COLUMN]

df_ASTD_UN = df_ASTD_UN[df_ASTD_UN[LABEL_COLUMN]!= 'OBJ']

train_ASTD_UN, test_ASTD_UN = train_test_split(
    df_ASTD_UN, test_size=0.2, random_state=42
)

label_list_ASTD_UN = list(df_ASTD_UN[LABEL_COLUMN].unique())
print(label_list_ASTD_UN)
print(df_ASTD_UN[LABEL_COLUMN].value_counts())

data_ASTD_UN = CustomDataset(
    "ASTD-Unbalanced", train_ASTD_UN, test_ASTD_UN, label_list_ASTD_UN
)

[0, 1]
0    4293
1    1426
Name: label, dtype: int64


In [ ]:
import numpy as np
import torch
import random
import matplotlib.pyplot as plt
import copy


##import transformers
##from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
##from transformers import AutoTokenizer, AutoModel


from arabert.preprocess import ArabertPreprocessor
from sklearn.metrics import (accuracy_score, classification_report,
                             confusion_matrix, f1_score, precision_score,
                             recall_score)
from torch.utils.data import DataLoader, Dataset
from transformers import (AutoConfig, AutoModelForSequenceClassification,
                          AutoTokenizer, BertTokenizer, Trainer,
                          TrainingArguments)
from transformers.data.processors.utils import InputFeatures
import transformers
import torch.optim as optim
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup


In [ ]:
# select a model from the huggingface modelhub https://huggingface.co/models?language=ar
#model_name = 'aubmindlab/bert-base-arabertv02-twitter'
#model_name='UBC-NLP/MARBERT'
model_name='aubmindlab/aragpt2-mega-detector-long'
#model_name='aubmindlab/aragpt2-mega'
#model_name = 'aubmindlab/araelectra-base-generator'

In [ ]:
arabic_prep = ArabertPreprocessor(model_name)
##arabic_prep =AutoTokenizer.from_pretrained(model_name)
selected_dataset=data_ASTD_UN
selected_dataset.train[DATA_COLUMN] = selected_dataset.train[DATA_COLUMN].apply(lambda x: arabic_prep.preprocess(x))
selected_dataset.test[DATA_COLUMN] = selected_dataset.test[DATA_COLUMN].apply(lambda x: arabic_prep.preprocess(x))

In [ ]:
tok = AutoTokenizer.from_pretrained(model_name)

In [ ]:
class ClassificationDataset(Dataset):
    def __init__(self, text, target, model_name, max_len, label_map):
      super(ClassificationDataset).__init__()
      """
      Args:
      text (List[str]): List of the training text
      target (List[str]): List of the training labels
      tokenizer_name (str): The tokenizer name (same as model_name).
      max_len (int): Maximum sentence length
      label_map (Dict[str,int]): A dictionary that maps the class labels to integer
      """
      self.text = text
      self.target = target
      self.tokenizer_name = model_name
      self.tokenizer = AutoTokenizer.from_pretrained(model_name)
      self.max_len = max_len
      self.label_map = label_map


    def __len__(self):
      return len(self.text)

    def __getitem__(self,item):
      text = str(self.text[item])
      text = " ".join(text.split())

      inputs = self.tokenizer(
          text,
          max_length=self.max_len,
          padding='max_length',
          truncation=True
      )
      return InputFeatures(**inputs,label=self.label_map[self.target[item]])

In [ ]:
max_len = 128
label_map = { v:index for index, v in enumerate(selected_dataset.label_list) }
print(label_map)

train_dataset = ClassificationDataset(
    selected_dataset.train[DATA_COLUMN].to_list(),
    selected_dataset.train[LABEL_COLUMN].to_list(),
    model_name,
    max_len,
    label_map
  )
test_dataset = ClassificationDataset(
    selected_dataset.test[DATA_COLUMN].to_list(),
    selected_dataset.test[LABEL_COLUMN].to_list(),
    model_name,
    max_len,
    label_map
  )

{0: 0, 1: 1}


In [ ]:
print(next(iter(train_dataset)))

Using pad_token, but it is not set yet.


ValueError: ignored

In [ ]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_name, return_dict=True, num_labels=len(label_map))

In [ ]:
from sklearn.metrics import classification_report
def compute_metrics(p): #p should be of type EvalPrediction
  global preds
  preds = np.argmax(p.predictions, axis=1)
  assert len(preds) == len(p.label_ids)

  macro_f1 = f1_score(p.label_ids,preds,average='macro')

  acc = accuracy_score(p.label_ids,preds)
  AAA=classification_report(p.label_ids, preds)
  print(classification_report(p.label_ids, preds))
  return {
      'macro_f1' : macro_f1,
      'accuracy': acc

  }

In [ ]:
def set_seed(seed=42):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  torch.backends.cudnn.deterministic=True
  torch.backends.cudnn.benchmark = False

In [ ]:
training_args = TrainingArguments(
    output_dir= "./train",
    adam_epsilon = 1e-8,
    learning_rate = 2e-5,
    fp16 = False, # enable this when using V100 or T4 GPU
    per_device_train_batch_size = 16, # up to 64 on 16GB with max len of 128
    per_device_eval_batch_size = 128,
    gradient_accumulation_steps = 2, # use this to scale batch size without needing more memory
    num_train_epochs= 5,
    warmup_ratio = 0,
    do_eval = True,
    evaluation_strategy = 'epoch',
    save_strategy = 'epoch',
    load_best_model_at_end = True, # this allows to automatically get the best model at the end based on whatever metric we want
    metric_for_best_model = 'macro_f1',
    greater_is_better = True,
    seed = 25
  )

set_seed(training_args.seed)

In [ ]:
trainer = Trainer(
    model = model_init(),
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

NameError: ignored

In [ ]:
trainer.train()

NameError: ignored